Baird Counter Example
---
To convert them to semi-gradient form, we simply replace the update to an array `(V or Q)` to an update to a weight vector `(w)`, using the approximate value function (`vˆ` or `qˆ`) and its gradient. Many of these algorithms use the per-step `importance sampling ratio`:

<img src="rho.png" width="200">

<img src="weights_update.png" width="300">

---
The dashed action takes the system to one of the six upper states with equal probability, whereas the solid action takes the system to the seventh state. The `behavior policy b` selects the dashed and solid actions with probabilities `6/7` and `1/7`, so that the next-state distribution under it is uniform (the same for all nonterminal states), which is also the starting distribution for each episode. The target policy `π` always takes the solid action, and so the on-policy distribution (for `π`) is concentrated in the seventh state. The reward is `zero` on all transitions. The discount rate is `γ = 0.99`.

<img src="Baird.png" width="600">

- [Target Policy & Behaviour Policy](https://stats.stackexchange.com/questions/410131/in-rl-why-using-a-behavior-policy-instead-of-the-target-policy-for-an-episode-i)
- [Target Policy & Behaviour Policy2](https://stats.stackexchange.com/questions/237085/how-to-correctly-compute-rho-in-reinforcement-learning-with-importance-sampli?rq=1)

In [1]:
import numpy as np

In [12]:
STATES = range(7)

In [79]:
class Baird:
    
    def __init__(self, gamma=0.99, alpha=0.01):
        self.state = np.random.choice(STATES)
        self.prob = 1.0/7
        self.actions = ["solid", "dash"]
        self.gamma = gamma
        self.alpha = alpha
        
        self.features = np.zeros((len(STATES), 8))  # n_states x n_weights (this is the representation of states)
        for i in range(len(STATES)):
            if i == 6:
                self.features[i, -2] = 1
                self.features[i, -1] = 2
            else:
                self.features[i, i] = 2
                self.features[i, -1] = 1
        
        self.weights = np.ones(8)
        self.weights[-2] = 10
        
    def chooseAction(self):
        if np.random.binomial(1, self.prob) == 1:
            action = "solid"
        else:
            action = "dash"
        return action
    
    def takeAction(self, action):
        if action == "solid":
            nxtState = 6
        else:
            nxtState = np.random.choice(STATES[:-1])
        return nxtState
    
    def value(self, state):
        v = np.dot(ba.features[state, :], ba.weights)
        return v
    
    def update(self, state, delta):
        self.weights += delta*self.features[state, :]
    
    def run(self, rounds=100):
        reward = 0
        for _ in range(rounds):
            action = self.chooseAction()
            nxtState = self.takeAction(action)
            
            if action == "dash":
                rho = 0
            else:
                rho = 1/self.prob
#             rho = 1
            
            delta = reward + self.gamma*self.value(nxtState) - self.value(self.state)
            delta *= self.alpha*rho
            self.update(self.state, delta)
                        
            self.state = nxtState

In [80]:
ba = Baird()
print(ba.features)
print()
print(ba.weights)

[[2. 0. 0. 0. 0. 0. 0. 1.]
 [0. 2. 0. 0. 0. 0. 0. 1.]
 [0. 0. 2. 0. 0. 0. 0. 1.]
 [0. 0. 0. 2. 0. 0. 0. 1.]
 [0. 0. 0. 0. 2. 0. 0. 1.]
 [0. 0. 0. 0. 0. 2. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 2.]]

[ 1.  1.  1.  1.  1.  1. 10.  1.]


In [81]:
ba = Baird()
ba.run(rounds=1000)

In [82]:
ba.weights

array([ 98.66944559,  87.82605498,  83.72652029,  96.42341865,
       113.56688306,  83.48255486,   5.62034809, 271.08813489])